In [ ]:

import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
df=pd.read_csv("/kaggle/input/heart-disease-uci/heart.csv")

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
import seaborn as sns

In [ ]:
sns.countplot('target',data=df)

In [ ]:
df.head()

In [ ]:
a = pd.get_dummies(df['sex'], prefix = "sex")
b = pd.get_dummies(df['cp'], prefix = "cp")
c = pd.get_dummies(df['fbs'], prefix = "fbs")
d = pd.get_dummies(df['restecg'],prefix='restecg')
e = pd.get_dummies(df['exang'],prefix='exang')
f = pd.get_dummies(df['slope'],prefix='slope')
g = pd.get_dummies(df['thal'],prefix='thal')
h = pd.get_dummies(df['ca'],prefix='ca')

In [ ]:
df2 = [df, a, b, c,d,e,f,g,h]
df = pd.concat(df2, axis = 1)
df.head()

In [ ]:
pd.set_option('display.max_columns', None)

df = df.drop(columns = ['cp', 'thal', 'slope','sex','fbs','restecg','ca','exang'])
df.head()

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz 
from sklearn.metrics import roc_curve, auc 
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 

In [ ]:
x=df.drop('target',axis=1)

In [ ]:
y=df['target']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)


In [ ]:

x_train = x_train.values
x_test = x_test.values
y_train = y_train.values
y_test = y_test.values

In [ ]:
x_train

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

classifiers = {
    "LogisiticRegression": LogisticRegression(),
    "KNearest": KNeighborsClassifier(),
    "Support Vector Classifier": SVC(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "RandomForestClassifier":RandomForestClassifier()
}

In [ ]:
import warnings
warnings.filterwarnings("ignore")


In [ ]:
from sklearn.model_selection import cross_val_score


for key, classifier in classifiers.items():
    classifier.fit(x_train, y_train)
    training_score = cross_val_score(classifier, x_train, y_train, cv=5)
    print("Classifiers: ", classifier.__class__.__name__, "Has a training score of", round(training_score.mean(), 2) * 100, "% accuracy score")

In [ ]:
from sklearn.model_selection import GridSearchCV


# Logistic Regression 
log_reg_params = {"penalty": ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}



grid_log_reg = GridSearchCV(LogisticRegression(), log_reg_params)
grid_log_reg.fit(x_train, y_train)
#We automatically get the logistic regression with the best parameters.
log_reg = grid_log_reg.best_estimator_

knears_params = {"n_neighbors": list(range(2,5,1)), 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']}

grid_knears = GridSearchCV(KNeighborsClassifier(), knears_params)
grid_knears.fit(x_train, y_train)
# KNears best estimator
knears_neighbors = grid_knears.best_estimator_

# # Support Vector Classifier
svc_params = {'C': [0.5, 0.7, 0.9, 1], 'kernel': ['rbf', 'poly', 'sigmoid', 'linear']}
grid_svc = GridSearchCV(SVC(), svc_params)
grid_svc.fit(x_train, y_train)

# # SVC best estimator
svc = grid_svc.best_estimator_

# # DecisionTree Classifier
tree_params = {"criterion": ["gini", "entropy"], "max_depth": list(range(2,4,1)), 
              "min_samples_leaf": list(range(5,7,1))}
grid_tree = GridSearchCV(DecisionTreeClassifier(), tree_params)
grid_tree.fit(x_train, y_train)

# # tree best estimator
tree_clf = grid_tree.best_estimator_

##random forest classifier
rfc=RandomForestClassifier(random_state=42)

param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid)
CV_rfc.fit(x_train, y_train)
random_tree_clf=CV_rfc.best_estimator_

In [ ]:
# random_tree_clf=CV_rfc.best_params_

In [ ]:
log_reg_score = cross_val_score(log_reg, x_train, y_train, cv=5)
print('Logistic Regression Cross Validation Score: ', round(log_reg_score.mean() * 100, 2).astype(str) + '%')


knears_score = cross_val_score(knears_neighbors, x_train, y_train, cv=5)
print('Knears Neighbors Cross Validation Score', round(knears_score.mean() * 100, 2).astype(str) + '%')
svc_score = cross_val_score(svc, x_train, y_train, cv=5)
print('Support Vector Classifier Cross Validation Score', round(svc_score.mean() * 100, 2).astype(str) + '%')

tree_score = cross_val_score(tree_clf, x_train, y_train, cv=5)
print('DecisionTree Classifier Cross Validation Score', round(tree_score.mean() * 100, 2).astype(str) + '%')
random_forest_score = cross_val_score(random_tree_clf, x_train, y_train, cv=5)
print('Random forest Classifier Cross Validation Score', round(random_forest_score.mean() * 100, 2).astype(str) + '%')

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_val_predict

log_reg_pred = cross_val_predict(log_reg, x_train, y_train, cv=5,
                             method="decision_function")

knears_pred = cross_val_predict(knears_neighbors, x_train, y_train, cv=5)

svc_pred = cross_val_predict(svc, x_train, y_train, cv=5,
                             method="decision_function")

tree_pred = cross_val_predict(tree_clf, x_train, y_train, cv=5)
random_forest_pred = cross_val_predict(random_tree_clf, x_train, y_train, cv=5)


In [ ]:
from sklearn.metrics import roc_auc_score

print('Logistic Regression: ', roc_auc_score(y_train, log_reg_pred))
print('KNears Neighbors: ', roc_auc_score(y_train, knears_pred))
print('Support Vector Classifier: ', roc_auc_score(y_train, svc_pred))
print('Decision Tree Classifier: ', roc_auc_score(y_train, tree_pred))
print('Random Forest Classifier: ', roc_auc_score(y_train, random_forest_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
y_pred_log_reg = log_reg.predict(x_test)

y_pred_knear = knears_neighbors.predict(x_test)
y_pred_svc = svc.predict(x_test)
y_pred_tree = tree_clf.predict(x_test)
y_pred_forest = random_tree_clf.predict(x_test)

log_reg_cf = confusion_matrix(y_test, y_pred_log_reg)
kneighbors_cf = confusion_matrix(y_test, y_pred_knear)
svc_cf = confusion_matrix(y_test, y_pred_svc)
tree_cf = confusion_matrix(y_test, y_pred_tree)
forest_cf = confusion_matrix(y_test, y_pred_forest)


fig, ax = plt.subplots(2, 2,figsize=(22,12))


sns.heatmap(log_reg_cf, ax=ax[0][0], annot=True, cmap=plt.cm.copper)
ax[0, 0].set_title("Logistic Regression \n Confusion Matrix", fontsize=14)
ax[0, 0].set_xticklabels(['', ''], fontsize=14, rotation=90)
ax[0, 0].set_yticklabels(['', ''], fontsize=14, rotation=360)

sns.heatmap(kneighbors_cf, ax=ax[0][1], annot=True, cmap=plt.cm.copper)
ax[0][1].set_title("KNearsNeighbors \n Confusion Matrix", fontsize=14)
ax[0][1].set_xticklabels(['', ''], fontsize=14, rotation=90)
ax[0][1].set_yticklabels(['', ''], fontsize=14, rotation=360)

sns.heatmap(svc_cf, ax=ax[1][0], annot=True, cmap=plt.cm.copper)
ax[1][0].set_title("Suppor Vector Classifier \n Confusion Matrix", fontsize=14)
ax[1][0].set_xticklabels(['', ''], fontsize=14, rotation=90)
ax[1][0].set_yticklabels(['', ''], fontsize=14, rotation=360)

sns.heatmap(tree_cf, ax=ax[1][1], annot=True, cmap=plt.cm.copper)
ax[1][1].set_title("DecisionTree Classifier \n Confusion Matrix", fontsize=14)
ax[1][1].set_xticklabels(['', ''], fontsize=14, rotation=90)
ax[1][1].set_yticklabels(['', ''], fontsize=14, rotation=360)

# sns.heatmap(forest_cf, ax=ax[2][0], annot=True, cmap=plt.cm.copper)
# ax[2][0].set_title("Random forest Classifier \n Confusion Matrix", fontsize=14)
# ax[2][0].set_xticklabels(['', ''], fontsize=14, rotation=90)
# ax[2][0].set_yticklabels(['', ''], fontsize=14, rotation=360)


plt.show()

In [ ]:
sns.heatmap(forest_cf,annot=True, cmap=plt.cm.copper)

In [ ]:
from sklearn.metrics import classification_report


print('Logistic Regression:')
print(classification_report(y_test, y_pred_log_reg))

print('KNears Neighbors:')
print(classification_report(y_test, y_pred_knear))

print('Support Vector Classifier:')
print(classification_report(y_test, y_pred_svc))

print('Support Vector Classifier:')
print(classification_report(y_test, y_pred_tree))
print('random forest Support Vector Classifier:')
print(classification_report(y_test, y_pred_forest))

As logistic regression is working best among the given model. So, we are using logistic regression as our model

In [ ]:
log_fpr, log_tpr, log_thresold = roc_curve(y_train, log_reg_pred)
knear_fpr, knear_tpr, knear_threshold = roc_curve(y_train, knears_pred)
svc_fpr, svc_tpr, svc_threshold = roc_curve(y_train, svc_pred)
tree_fpr, tree_tpr, tree_threshold = roc_curve(y_train, tree_pred)
random_forest_fpr, random_forest_tpr, random_forest_threshold = roc_curve(y_train, random_forest_pred)
def graph_roc_curve_multiple(log_fpr, log_tpr, knear_fpr, knear_tpr, svc_fpr, svc_tpr, tree_fpr, tree_tpr,random_forest_fpr,random_forest_tpr):
    plt.figure(figsize=(16,8))
    plt.title('ROC Curve \n Top 4 Classifiers', fontsize=18)
    plt.plot(log_fpr, log_tpr, label='Logistic Regression Classifier Score: {:.4f}'.format(roc_auc_score(y_train, log_reg_pred)))
    plt.plot(knear_fpr, knear_tpr, label='KNears Neighbors Classifier Score: {:.4f}'.format(roc_auc_score(y_train, knears_pred)))
    plt.plot(svc_fpr, svc_tpr, label='Support Vector Classifier Score: {:.4f}'.format(roc_auc_score(y_train, svc_pred)))
    plt.plot(tree_fpr, tree_tpr, label='Decision Tree Classifier Score: {:.4f}'.format(roc_auc_score(y_train, tree_pred)))
    plt.plot(random_forest_fpr, random_forest_tpr, label='Random forest Classifier Score: {:.4f}'.format(roc_auc_score(y_train, random_forest_pred)))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([-0.01, 1, 0, 1])
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate', fontsize=16)
    plt.annotate('Minimum ROC Score of 50% \n (This is the minimum score to get)', xy=(0.5, 0.5), xytext=(0.6, 0.3),
                arrowprops=dict(facecolor='#6E726D', shrink=0.05),
                )
    plt.legend()
    
graph_roc_curve_multiple(log_fpr, log_tpr, knear_fpr, knear_tpr, svc_fpr, svc_tpr, tree_fpr, tree_tpr,random_forest_fpr,random_forest_tpr)
plt.show()

In [ ]:

log_reg.predict([x_test[1]])[0]

In [ ]:
y_test[1]

In [ ]:
data = [{'age': 63, 'sex': 1, 'cp':3,'trestbps':145, 'chol':233, 'fbs':1, 'restecg':0, 'thalach':150, 'exang':0, 'oldpeak':2.3, 'slope':0, 'ca':0, 'thal':1}]

In [ ]:
df1 = pd.DataFrame(data)
df1

In [ ]:
# df1 = df1.reindex(columns = df.columns, fill_value=0)

In [ ]:
df2 = pd.DataFrame(data)
df2

In [ ]:
df3=pd.read_csv("../input/heart-disease-uci/heart.csv")

In [ ]:
train_objs_num = len(df3)
dataset = pd.concat(objs=[df3, df2], axis=0)
a = pd.get_dummies(dataset['sex'], prefix = "sex")
b = pd.get_dummies(dataset['cp'], prefix = "cp")
c = pd.get_dummies(dataset['fbs'], prefix = "fbs")
d = pd.get_dummies(dataset['restecg'],prefix='restecg')
e = pd.get_dummies(dataset['exang'],prefix='exang')
f = pd.get_dummies(dataset['slope'],prefix='slope')
g = pd.get_dummies(dataset['thal'],prefix='thal')
h = pd.get_dummies(dataset['ca'],prefix='ca')
df2 = [dataset, a, b, c,d,e,f,g,h]
dataset = pd.concat(df2, axis = 1)
dataset = dataset.drop(columns = ['cp', 'thal', 'slope','sex','fbs','restecg','ca','exang'])

# dataset = pd.get_dummies(dataset)
train = dataset[:train_objs_num].copy()
test = dataset[train_objs_num:].copy()

In [ ]:
test=test.drop('target',axis=1)
test.head()

In [ ]:
test.shape

In [ ]:
log_reg.predict(test)[0]

In [ ]:
df.head(1)

In [ ]:
grid_log_reg.best_params_

In [ ]:
from sklearn.pipeline import Pipeline


In [ ]:
model_pipeline = Pipeline(steps=[('log_reg',LogisticRegression(C=1,penalty='l2'))])

In [ ]:
model_pipeline.fit(x_train,y_train)

In [ ]:
from pickle import dump
dump(model_pipeline, open('heart_model.pkl', 'wb'))

In [ ]:
from pickle import load
model = load(open('./heart_model.pkl', 'rb'))
predictions = model.predict(x_test)


In [ ]:
print('Logistic Regression:')
print(classification_report(y_test, predictions))